In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np

from dynamicslib.consts import muEM
from dynamicslib.continuation import find_bif, arclen_cont
from dynamicslib.common import prop_ic, get_JC_tf, eom,f_df_CR3_single, coupled_stm_eom
from dynamicslib.targeter import dc_arclen, dc_square
from dynamicslib.integrator import dop853
from tqdm.auto import tqdm
from dynamicslib.plotting import plotly_display, to_eci
from dynamicslib.common_targetters import spatial_perpendicular, spatial_period_fixed

family = "L1 Halo"

root = os.getcwd()
file = f"{root}/database/{family}.csv"
db = pd.read_csv(file).set_index("Index")

In [ ]:
targetter = spatial_perpendicular(1e-10, muEM)
func = targetter.f_df_stm


def X2xtf(X):
    return targetter.get_x0(X), targetter.get_tf(X)

In [ ]:
if "Initial z" not in db.columns:
    db["Initial z"] = 0.0
# at 1127?
Xm0 = db[["Initial x", "Initial z", "Initial vy", "Period"]].iloc[140].values
Xm1 = db[["Initial x", "Initial z", "Initial vy", "Period"]].iloc[139].values
Xm0[-1] /= 2
Xm1[-1] /= 2

In [ ]:
# vals = db[["Initial x", "Initial z", "Initial vy", "Period"]].values
# dprods = [np.dot(vals[j + 2] - vals[j + 1], vals[j + 1] - vals[j]) for j in range(len(vals) - 2)]
# plt.plot(dprods)

In [ ]:
# x0 = get_x0(X0)
# dop853(
#     coupled_stm_eom,
#     (0.0, 5.0 / 2),
#     np.append(x0, np.eye(6).flatten()),
#     int_tol,
#     int_tol,
#     args=(mu,),
# )

In [ ]:
X0, _ = find_bif(
    Xm0,
    func,
    -(Xm0 - Xm1),
    5e-3,
    targ_tol=1e-11,
    bisect_tol=1e-11,
    bif_type=(2,),
    debug=True,
)

In [ ]:
# # switch direction!
x0, tf = targetter.get_x0(X0), targetter.get_tf(X0)
ts, xs, _ = dop853(eom, (0, tf / 2), x0, 1e-14, 0)
xs[1, -1] = 0
X0 = targetter.get_X(xs[:, -1], ts[-1] * 2)

In [ ]:
X1 = X0.copy()
X1[-1]*=2
f, df, stm = func(X1)
svd = np.linalg.svd(df)
tangent = svd.Vh[-2]
tangent

In [ ]:
X, dF, stm = dc_arclen(
    X1, tangent, func, 1e-4, 1e-10, max_iter=100, debug=True, modified=True
)
X

In [ ]:
Xsp1, esp1 = arclen_cont(
    X1, func, tangent, 1e-4, 0.05, 1e-9, exact_tangent=True, max_iter=200
)

In [ ]:
# Xsm1, esm1 = arclen_cont(X1, func, -tangent2, 1e-3, 0.1, 1e-10, use_exact_tangent=True)

In [ ]:
Xsp2, esp2 = arclen_cont(
    Xsp1[-1], func, Xsp1[-1] - Xsp1[-2], 1e-3, 0.2, 1e-9, max_iter=250
)

In [ ]:
Xsp3, esp3 = arclen_cont(
    Xsp2[-1], func, Xsp2[-1] - Xsp2[-2], 1e-3, 0.1, 1e-8, max_iter=250
)

In [ ]:
Xsp4, esp4 = arclen_cont(
    Xsp3[-1], func, Xsp3[-1] - Xsp3[-2], 1e-3, 0.03, 1e-8, max_iter=250
)

In [ ]:
Xsp5, esp5 = arclen_cont(
    Xsp4[-1], func, Xsp4[-1] - Xsp4[-2], 5e-4, 0.05, 1e-8, max_iter=250, fudge=0.5
)

In [ ]:
Xsp6, esp6 = arclen_cont(
    Xsp5[-1], func, Xsp5[-1] - Xsp5[-2], 5e-4, 0.05, 1e-8, max_iter=250, fudge=0.5
)

In [ ]:
# f, df, stm = func(Xsp10[-1])
# svd = np.linalg.svd(df)
# tngt = svd.Vh[-2]

In [ ]:
# _, df, _ = func(Xsp2[-1])
# tangent = np.linalg.svd(df).Vh[-1]

# Xsp3, esp3 = arclen_to_fail(Xsp2[-1], func, tangent, 5e-3, 1e-8, 30)

In [ ]:
Xsp = [*Xsp1, *Xsp2[1:]]
esp = [*esp1, *esp2[1:]]
Xsm = []
esm = []

param_names = [
    "Index",
    "Initial x",
    "Initial z",
    "Initial vy",
    "Period",
    "Jacobi Constant",
    "Stability Index",
    "Eig1",
    "Eig2",
    "Eig3",
    "Eig4",
    "Eig5",
    "Eig6",
]

xyzs = []
data = []


iterator = list(enumerate(zip(Xsm, esm)))[::-1]
for i, (X, evals) in tqdm(iterator):
    Xcp = X.copy()
    xyzs.append(prop_ic(Xcp, X2xtf, int_tol=1e-13))
    args = np.argsort(np.abs(evals))
    stabind = np.max([(np.abs(lam) + 1 / np.abs(lam)) / 2 for lam in evals])
    jc, period = get_JC_tf(Xcp, X2xtf)
    data.append([-i, *Xcp[:-1], period, jc, stabind, *evals])

iterator = list(enumerate(zip(Xsp, esp)))
for i, (X, evals) in tqdm(iterator):
    Xcp = X.copy()
    # Xcp[-1]
    xyzs.append(prop_ic(Xcp, X2xtf, int_tol=1e-13))
    args = np.argsort(np.abs(evals))
    stabind = np.max([(np.abs(lam) + 1 / np.abs(lam)) / 2 for lam in evals])
    jc, period = get_JC_tf(Xcp, X2xtf)
    data.append([i, *Xcp[:-1], period, jc, stabind, *evals])

In [ ]:
df = pd.DataFrame(data, columns=param_names).set_index("Index")
# df.to_csv("P2 DPO (spatial).csv")

In [ ]:
a = None
b = None
c = 5
plotly_display(xyzs[a:b:c], df[a:b:c], flip_vert=False)